<a href="https://colab.research.google.com/github/dorado7/Image-Classification/blob/main/MachineLearningwithDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
from torchvision import datasets
from torchvision import transforms as T
from torch.utils.data import DataLoader
import os
import torch.nn as nn
import torch.optim as optim
import torch
import torchvision
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import pathlib
from PIL import Image
from google.colab import drive
import glob

# uploaded = files.upload()

In [ ]:
drive.mount('/content/drive')

dataset_path = '/content/drive/MyDrive/BrainMRI_CW'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
# Define transformations
transforms = T.Compose([ # Call Compose using the alias
    T.Resize((150,150)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])

test_path = '/content/drive/MyDrive/BrainMRI_CW/Testing'
train_path = '/content/drive/MyDrive/BrainMRI_CW/Training'
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform=transforms),
    batch_size=256, shuffle=True # The batch size is according to the CPU memory ensure the allocated sizee is appropriate
)
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform=transforms),
    batch_size=256, shuffle=True
)

In [ ]:
#Check initial image size
""" needs to be completed """

' needs to be completed '

In [ ]:
#Categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['glioma', 'meningioma', 'notumor', 'pituitary']


In [ ]:
#CNN Network
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
#CNN Network
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()

        #Output size after convolution filter
       # ((w-f+2P)/s) +1

        #Input shape= (256,3,150,150)

        self.conv1=nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()

        self.pool=nn.MaxPool2d(kernel_size=2)

        self.conv2=nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        self.relu2=nn.ReLU()

        self.conv3=nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()

        self.fc=nn.Linear(in_features=32*75*75, out_features=num_classes)

    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)

        output=self.pool(output)

        output=self.conv2(output)
        output=self.relu2(output)

        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)

        output=output.view(-1,32*75*75)

        output=self.fc(output)
        return output

        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()

        self.pool=nn.MaxPool2d(kernel_size=2)

        self.conv2=nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        self.relu2=nn.ReLU()

        self.conv3=nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()

        self.fc=nn.Linear(in_features=32*75*75, out_features=num_classes)

        def forward(self,input):
            output=self.conv1(input)
            output=self.bn1(output)
            output=self.relu1(output)

            output=self.pool(output)

            output=self.conv2(output)
            output=self.relu2(output)

            output=self.conv3(output)
            output=self.bn3(output)
            output=self.relu3(output)

            output=output.view(-1,32*75*75)

            output=self.fc(output)

            return output



In [ ]:
model = ConvNet(num_classes=6).to(device)

In [ ]:
# Optimiser and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()

In [ ]:
num_epochs=10

In [ ]:
#calculating the size of training and testing set (check this)
train_count = len(glob.glob(train_path+'/**/*.jpg'))
test_count = len(glob.glob(test_path+'/**/*.jpg'))

In [ ]:
print(train_count, test_count)

5722 1311


In [ ]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(10):

    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0

    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        optimizer.zero_grad()
        outputs=model(images)
        loss=loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        #This will calculate the loss
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)


        #Expalin
        train_accuracy+=int(torch.sum(prediction==labels.data))

        #This will calculate the accuracy
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count

    #Evaluation and training on testing dataset
    model.eval()

    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

    test_accuracy=test_accuracy/test_count

    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(10.0723) Train Accuracy: 0.6022369800768962 Test Accuracy: 0.4988558352402746
Epoch: 1 Train Loss: tensor(1.6306) Train Accuracy: 0.8210415938483048 Test Accuracy: 0.6216628527841342
Epoch: 2 Train Loss: tensor(0.6111) Train Accuracy: 0.8841314225795176 Test Accuracy: 0.8535469107551488
Epoch: 3 Train Loss: tensor(0.2410) Train Accuracy: 0.9388325760223698 Test Accuracy: 0.8863463005339436
Epoch: 4 Train Loss: tensor(0.1959) Train Accuracy: 0.9369101712687872 Test Accuracy: 0.8779557589626239
Epoch: 5 Train Loss: tensor(0.1483) Train Accuracy: 0.9627752534078994 Test Accuracy: 0.9206712433257056
Epoch: 6 Train Loss: tensor(0.0979) Train Accuracy: 0.9772806710940231 Test Accuracy: 0.9466056445461479
Epoch: 7 Train Loss: tensor(0.0547) Train Accuracy: 0.9800768961901433 Test Accuracy: 0.9481311975591151
Epoch: 8 Train Loss: tensor(0.0443) Train Accuracy: 0.988465571478504 Test Accuracy: 0.9427917620137299
Epoch: 9 Train Loss: tensor(0.0271) Train Accuracy: 0.9